# Capstone I Report: Machine Learning

This report is supposed to be a milestone report that summarize all the previous reports. But in last meeting my mentor said there is no pass/fail option for the reports, I can basically write whatever I want (not ture).

Anyway, I decide that my milestone report for capstone 1 would be a first-phase machine learning report. Based on the recommendations from the previous reports, in this report I would perform SVM (with several kernels), Decision Tree, Random Forests, and Neuro Network. 

The algorithm is first tuned based on train samples based on train-test split, then they are applied to test samples and the result is submitted to kaggle to see how well the models work.

## Get the training sample ready

In [1]:
import pandas as pd
import numpy as np
import sys
import re
df_ordered = pd.read_csv("../code/ordered.csv")
df_orderedtest = pd.read_csv("../code/ordered_test.csv")

In [2]:
mask = (df_ordered['WEvent'] == 1)

df_ordered['Before_Event'] = (pd.rolling_mean(df_ordered['Condition'], window=3).shift(-3) > 0)
df_ordered['After_Event'] = (pd.rolling_mean(df_ordered['Condition'], window=3).shift(1) > 0)

mask = (df_orderedtest['WEvent'] == 1)

df_orderedtest['Before_Event'] = (pd.rolling_mean(df_orderedtest['Condition'], window=3).shift(-3) > 0)
df_orderedtest['After_Event'] = (pd.rolling_mean(df_orderedtest['Condition'], window=3).shift(1) > 0)

/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  app.launch_new_instance()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()


In [3]:
df_ordered['year'] = pd.to_datetime(df_ordered['date'], infer_datetime_format=True).dt.year
df_ordered['month'] = pd.to_datetime(df_ordered['date'], infer_datetime_format=True).dt.month
df_ordered['weekday'] = pd.to_datetime(df_ordered['date'], infer_datetime_format=True).dt.weekday

df_orderedtest['year'] = pd.to_datetime(df_orderedtest['date'], infer_datetime_format=True).dt.year
df_orderedtest['month'] = pd.to_datetime(df_orderedtest['date'], infer_datetime_format=True).dt.month
df_orderedtest['weekday'] = pd.to_datetime(df_orderedtest['date'], infer_datetime_format=True).dt.weekday


## drop unused columns

df_ordered = df_ordered.drop(['station_nbr','tmax','tmin','depart','dewpoint','wetbulb','heat','cool','sunrise','sunset','codesum','snowfall','stnpressure','sealevel','resultspeed'],axis=1)
df_orderedtest = df_orderedtest.drop(['station_nbr','tmax','tmin','depart','dewpoint','wetbulb','heat','cool','sunrise','sunset','codesum','snowfall','stnpressure','sealevel','resultspeed'],axis=1)

## get_dummies

df_ordered = pd.get_dummies(df_ordered, columns=['year','month','weekday'])
df_ordered = df_ordered.drop(['Unnamed: 0','date'],axis=1)

df_orderedtest = pd.get_dummies(df_orderedtest, columns=['year','month','weekday'])

df_orderedtest['id'] = df_orderedtest['store_nbr'].astype(str) + '_' + df_orderedtest['item_nbr'].astype(str) + '_' + df_orderedtest['date'].astype(str)
df_orderedtest = df_orderedtest.set_index('id')

df_orderedtest = df_orderedtest.drop(['Unnamed: 0','date'],axis=1)
## train test split

In [4]:
item_nbr_list = list(set(df_ordered['item_nbr']))
store_nbr_list = list(set(df_ordered['store_nbr']))

df_orderedtest.insert(12, 'year_2012', 0)
df_orderedtest['units'] = 0

In [5]:
False == 0

True

## SVM

In [6]:
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from sklearn.svm import SVR

for store in store_nbr_list:
    for item in item_nbr_list:
        mask = ((df_ordered['item_nbr'] == item) & (df_ordered['store_nbr'] == store))
        y_train = df_ordered.loc[mask,'units']
        X_train = df_ordered.loc[mask,df_ordered.columns != 'units']
        
        mask = ((df_orderedtest['item_nbr'] == item) & (df_orderedtest['store_nbr'] == store))
        X_test = df_orderedtest.loc[mask,df_orderedtest.columns != 'units']
        
        if X_test.empty == 0:
       
            scaling = MinMaxScaler(feature_range=(0,1)).fit(X_train)
            X_train = scaling.transform(X_train)
            X_test = scaling.transform(X_test)
        
            svr = SVR()
            svr.fit(X_train, y_train)
            df_orderedtest.loc[mask,'units'] = svr.predict(X_test)
        
    print store
        
result = df_orderedtest['units']

print result
        
        
        
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
id
1_1_2013-06-04       0.0
1_1_2013-06-05       0.0
1_1_2013-06-06       0.0
1_1_2013-06-07       0.0
1_1_2013-06-08       0.0
1_1_2013-06-09       0.0
1_1_2013-06-10       0.0
1_1_2013-06-28       0.0
1_1_2013-06-29       0.0
1_1_2013-06-30       0.0
1_1_2013-07-01       0.0
1_1_2013-07-02       0.0
1_1_2013-07-03       0.0
1_1_2013-07-04       0.0
1_1_2013-07-20       0.0
1_1_2013-07-21       0.0
1_1_2013-07-22       0.0
1_1_2013-07-23       0.0
1_1_2013-07-24       0.0
1_1_2013-07-25       0.0
1_1_2013-07-26       0.0
1_1_2013-07-30       0.0
1_1_2013-07-31       0.0
1_1_2013-08-01       0.0
1_1_2013-08-02       0.0
1_1_2013-08-03       0.0
1_1_2013-08-04       0.0
1_1_2013-08-05       0.0
1_1_2013-08-06       0.0
1_1_2013-08-07       0.0
                    ... 
45_111_2014-04-02    0.0
45_111_2014-06-23    0.0
45_111_2014-06-24    0.0
45_111_2014-06-25    

In [7]:
result = result.reset_index()

In [12]:
mask = (result['units'] < 0)
result.loc[mask,'units'] = 0

In [13]:
result.to_csv("../code/result.csv",sep=",",index=False)

## Decision Tree and Random Forest

In [ ]:
## apply decision tree and random forest

## Neuro Network

## Kaggle Performance

### What if I submit a file predicting everything to zero?

### Get the testing samples ready

### Using models